# Tutorial: Filtering Documents with Metadata

Mặc dù các kỹ thuật truy xuất mới rất tuyệt vời nhưng đôi khi bạn chỉ biết rằng bạn muốn thực hiện tìm kiếm trên một nhóm tài liệu cụ thể trong kho tài liệu của mình. Đây có thể là bất cứ thứ gì từ tất cả các tài liệu có liên quan đến một người dùng cụ thể hoặc được xuất bản sau một ngày nhất định , v.v. Lọc siêu dữ liệu rất hữu ích trong những tình huống này. Trong hướng dẫn này, chúng tôi sẽ tạo một số tài liệu đơn giản chứa thông tin về Haystack, trong đó siêu dữ liệu bao gồm thông tin về phiên bản Haystack mà thông tin liên quan đến. Sau đó, chúng tôi sẽ lọc siêu dữ liệu để đảm bảo rằng chúng tôi chỉ trả lời câu hỏi dựa trên thông tin về Haystack 2.0.

In [2]:
# %%bash

# pip install haystack-ai

## Preparing Documents

In [3]:
from datetime import datetime

from haystack import Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever

documents = [
    Document(
        content="Use pip to install a basic version of Haystack's latest release: pip install farm-haystack. All the core Haystack components live in the haystack repo. But there's also the haystack-extras repo which contains components that are not as widely used, and you need to install them separately.",
        meta={"version": 1.15, "date": datetime(2023, 3, 30)},
    ),
    Document(
        content="Use pip to install a basic version of Haystack's latest release: pip install farm-haystack[inference]. All the core Haystack components live in the haystack repo. But there's also the haystack-extras repo which contains components that are not as widely used, and you need to install them separately.",
        meta={"version": 1.22, "date": datetime(2023, 11, 7)},
    ),
    Document(
        content="Use pip to install only the Haystack 2.0 code: pip install haystack-ai. The haystack-ai package is built on the main branch which is an unstable beta version, but it's useful if you want to try the new features as soon as they are merged.",
        meta={"version": 2.0, "date": datetime(2023, 12, 4)},
    ),
]
document_store = InMemoryDocumentStore(bm25_algorithm="BM25Plus")
document_store.write_documents(documents=documents)


/Users/ngohongthai/miniconda3/envs/nmt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3

## Building a Document Search Pipeline

In [4]:
from haystack import Pipeline

pipeline = Pipeline()
pipeline.add_component(instance=InMemoryBM25Retriever(document_store=document_store), name="retriever")


## Do Metadata Filtering

In [5]:
query = "Haystack installation"
pipeline.run(data={"retriever": {"query": query, "filters": {"field": "meta.version", "operator": ">", "value": 1.21}}})


Ranking by BM25...: 100%|██████████| 2/2 [00:00<00:00, 2049.00 docs/s]


{'retriever': {'documents': [Document(id=b53625c67fee5ba5ac6dc86e7ca0adff567bf8376e86ae4b3fc6f6f858ccf1e5, content: 'Use pip to install a basic version of Haystack's latest release: pip install farm-haystack[inference...', meta: {'version': 1.22, 'date': datetime.datetime(2023, 11, 7, 0, 0)}, score: 1.1808764808011376),
   Document(id=8ac1f8119bdec5c898d5a5c69f49ff47f64056bce1a0f95073e34493bbaf9354, content: 'Use pip to install only the Haystack 2.0 code: pip install haystack-ai. The haystack-ai package is b...', meta: {'version': 2.0, 'date': datetime.datetime(2023, 12, 4, 0, 0)}, score: 1.0867343954443756)]}}

In [6]:
query = "Haystack installation"
pipeline.run(
    data={
        "retriever": {
            "query": query,
            "filters": {
                "operator": "AND",
                "conditions": [
                    {"field": "meta.version", "operator": ">", "value": 1.21},
                    {"field": "meta.date", "operator": ">", "value": datetime(2023, 11, 7)},
                ],
            },
        }
    }
)


Ranking by BM25...: 100%|██████████| 1/1 [00:00<00:00, 7133.17 docs/s]


{'retriever': {'documents': [Document(id=8ac1f8119bdec5c898d5a5c69f49ff47f64056bce1a0f95073e34493bbaf9354, content: 'Use pip to install only the Haystack 2.0 code: pip install haystack-ai. The haystack-ai package is b...', meta: {'version': 2.0, 'date': datetime.datetime(2023, 12, 4, 0, 0)}, score: 1.8483924814931876)]}}